# Building a Routing Application


In today's exercise, we're going to use OpenRouteServices, a free routing and network analysis service, run on open-source software. We have two tasks for the day: 1) to write Python code that will permit us to collect routes from one point to another, and 2) to write Python code that will permit us to collect isochrones for individual points based on transit mode. Isochrones are polygons that depict network distance---in other words, how far can I travel along a road network given a certain amount of time?

As such, the first task is to get an [OpenRouteServices API key](https://openrouteservice.org/). The free plan is a bit constrained (500 isochrone requests per day, 2000 directions requests per day), but it will be totally fine for our purposes! Once you've created an account, copy your API key---you'll need it later!

## Create a New Virtual Environment

We begin by creating a new virtual environment for our routing work. As a reminder, a virtual environment is a self-contained Python environment, generally built to accommodate a specific project. We use these so that we're not endlessly installing pacakges into a single bloated environment; instead, we work only with those packages we need. 

First, we want to navigate to the folder to which we downloaded the exercise data. In a terminal (macOS/Linux) or command prompt (Windows) window type...

```sh
cd path/to/your/data/
```

Once you've changed your working directory, you'll want to create a new virtual environment using the `venv` utility, which is a module (hence, `-m`) of Python.

```sh
# Tells python 3 to run the venv module...
# ...and create a new virtual environment...
# ...in our current directory.
python3 -m venv venv/

# On Windows...
python -m venv venv\
```

We can activate this virtual environment by running the following at the terminal:

```sh
# On macOS or Linux...
. venv/bin/activate
# On Windows...
venv\Scripts\activate.bat
# The next line of your prompt should begin (venv)
```

Once we've activated our virtual environment, any packages we install will be specific to this environment---this is very useful for managing packages on a project-by-project basis. We're going to install the `openrouteservices` package, which is a fairly thin wrapper for the OpenRouteService API (by 'thin wrapper', I just mean that it provides convenience functions without contributing much in the way of added functionality). We're also going to install `python-dotenv` and `shapely`. The former allows us to store API keys and other secrets in a file that doesn't ever go anywhere near our git repository; we'll use the latter to store coordinates returned by OpenRouteService in what's called 'Well-Known Text'.

```sh
pip install openrouteservice jupyter python-dotenv shapely
```

## Import Modules

Run Jupyter from inside your virtual environment like this:

```sh
python -m jupyter notebook
```

You can work directly from this Jupyter Notebook, or create your own (the latter is probably a better learning exercise---typing=muscle memory=retention). Either way, we first need to import the relevant `client` object from the `openrouteservice` library. We're also going to import `pprint`, a Python utility function that parses and 'prettifies' complex data output.

In [1]:
from openrouteservice import client
from pprint import pprint

Note the very classic Python syntax here---we're not importing entire libraries! We're only importing the modules we need. Python prides itself on being extremely modular. This makes for programs that run very quickly, as scripts spend less time loading unused modules.

## Set up the OpenRouteService client

We'll then create a new instance of the OpenRouteService client object; in order to do this, we need to use the API key that we were given when we created a new OpenRouteService account. But we want to do this in a way that prevents others from finding our key. In modern programming, a surprising amount of work takes place in public in, for example, git repositories (e.g., [Github](https://github.com/), [Gitlab](https://gitlab.com/)). Your goal should be to __NEVER COMMIT YOUR KEY__ to your public repository. This means writing your code such that the key is read in from what's called an 'environment variable', instead of hard-coding it into your scripts.

### Store Your API key in an Environment Variable

The easiest way to store an environment variable is to create a new file called simply `.env` in your project directory and use a nifty package called `python-dotenv` to load your variables. If you're using a git repository, make sure you add `.env` to your `.gitignore` file!

Create a new `.env` file in your working directory that includes this line:

```sh
OSRKEY=yourapikey
```

Once you've set the environment variable, reading it in a Python script becomes fairly trivial:

In [42]:
import os
from dotenv import load_dotenv
load_dotenv()

osrkey = os.getenv('ORSKEY')

## Get a Single Route

We'll first create an instance of the OpenRouteService client object. You will simply pass your `osrkey` variable to its `key` parameter, like so:

In [40]:
ors = client.Client(key=osrkey)

Now, let's run a query. The MBTA is currently in the process of extending the Green Line into Union Square and, along a separate spur, into Medford. Say we want to know how accessible the new Union Square stop (`[-71.094038, 42.378790]`) will be to students at the Argenziano School (`[-71.099530, 42.379235]`) in Somerville.

We'll do this by passing a set of `kwargs` to the `directions` method of our client object (`ors`) using a Python dictionary. The OpenRouteService API is very well-documented; [you can find an inclusive list of all parameters that it accepts here](https://openrouteservice.org/dev/#/api-docs/directions). We want to know what the fastest route is between the Argenziano School and the T station, assuming that we are on foot. We also pass a parameter that requests a GeoJSON formatted output.

In [43]:
params = {'coordinates': [[-71.099530, 42.379235], 
                        [-71.094038, 42.378790]],
          'profile': 'foot-walking',
          'instructions': 'false',
          'format_out': 'geojson',
          'units': 'km'}
try:
    route = ors.directions(**params)
    pprint(route)
except Exception as err:
    pprint(err)

{'bbox': [-71.099533, 42.378807, -71.09427, 42.379696],
 'features': [{'bbox': [-71.099533, 42.378807, -71.09427, 42.379696],
               'geometry': {'coordinates': [[-71.099533, 42.379329],
                                            [-71.099521, 42.379329],
                                            [-71.099463, 42.379329],
                                            [-71.099292, 42.379341],
                                            [-71.098867, 42.379389],
                                            [-71.098639, 42.379412],
                                            [-71.098508, 42.379426],
                                            [-71.098436, 42.379434],
                                            [-71.097975, 42.379475],
                                            [-71.097894, 42.379483],
                                            [-71.097696, 42.379503],
                                            [-71.097427, 42.37953],
                                            [-7

The API returns a LineString detailing the route, alongside some metadata. Because this is already a GeoJSON file, we can simply write the result as a variable using the `dump` method from the `json` library.

In [ ]:
import json

json.dump(route, open('single_route.geojson', 'w'))

## Single Isochrone

In addition to returning routes, OpenRouteService can return what are called _isochrones_, or bounded areas accessible within a certain amount of time or less. This of course makes many assumptions about the mobility of the traveler, but it is nevertheless a useful metric.

We query for an isochrone in much the same way as we query for directions. Here, we are requesting the area accessible within 10 minutes (600 seconds) on foot from the new Green Line station in Union Square. Much like above, the isochrone service is well-documented, and I encourage you to spend some time [combing through the available parameters in the documentation](https://openrouteservice.org/dev/#/api-docs/v2/isochrones/{profile}/post).

In [41]:
params = {'locations': [[-71.094038, 42.378790]],
              'profile': 'foot-walking',
              'range_type': 'time',
              'range': [600]}

try:
    request = ors.isochrones(**params)
    pprint(request)
except Exception as err:
    pprint(err)
    

{'bbox': [-71.10367, 42.371606, -71.085581, 42.385323],
 'features': [{'geometry': {'coordinates': [[[-71.10367, 42.379126],
                                             [-71.103646, 42.378891],
                                             [-71.102889, 42.375047],
                                             [-71.102674, 42.374759],
                                             [-71.099021, 42.372008],
                                             [-71.095982, 42.371616],
                                             [-71.095011, 42.371606],
                                             [-71.094304, 42.371807],
                                             [-71.092831, 42.372513],
                                             [-71.088276, 42.374202],
                                             [-71.085756, 42.375166],
                                             [-71.085592, 42.375486],
                                             [-71.085581, 42.377326],
                                   

By default, this request returns a GeoJSON. This implies, again, that we can simply write the single request to a GeoJSON for use in e.g., QGIS.

In [ ]:
json.dump(request, open('single_isochrone.geojson', 'w'))

## Isochrones for Many Features

However, the above exercises are fairly trivial. Services like OpenRouteService become vastly more useful when we are able to do work in batches. This is fairly simple. The method below will approach this problem using only CSVs and (nearly) bare-minimum Python. No fancy or bloated libraries here!

We'll begin by simply reading the file I provided, `camberville_t.csv`. This is the location of curent T stops in Cambridge and Somerville from MassGIS; I've extended it to also include the planned Green Line Extension stops (which means that technically, the file includes a stop in Medford as well). Let's first make sure we are comfortable looping through rows in a CSV file; our approach will be to loop through and, for each feature, produce as isochrone that we append as a geometry!

We `open` our file in read mode (`'r'`). We then use the DictReader csv method to read the CSV as a Python dictionary.  We then iterate through its lines printing the values stored in the `STATION` column. Why do this instead of simply looping through rows? Good question! Because I like the ability to reference columns by name, which requires that we represent the CSV in a way that gives us access to headers. We could also do this using, for example, Pandas dataframes. But I'm trying to keep this as simple as possible.

In [28]:
# Import the csv library.
import csv

# Provide the location of the CSV file.
mbta_locs = 'camberville_t.csv'

# Open the file...
try:
    with open(mbta_locs, 'r') as file:
        # ...and read its contents as a dictionary.
        reader = csv.DictReader(file)
        # Iterate over the CSV rows...
        for line in reader:
            # ...and print the value of the 'STATION' column.
            print(line['STATION'])
except FileNotFoundError:
    print(f'{mbta_locs} does not exist')

Harvard
Kendall/MIT
Davis
Sullivan Square
Community College
Lechmere
Alewife
Porter
Central
Assembly
Union Square
East Somerville
Gilman Square
Magoun Square
Ball Square
College Ave


We then want to use the parameter dictionary, just like above. This time, though, we're going to give it `X` and `Y` values stored in our CSV. Note that we also sleep for a little more than a second---this is because there's a rate limit on the API, and if we go too quickly, we'll get booted! We also assign the resulting geometry to the `geom` column of the CSV line---specifically, we use the `shapely` library to convert it to well-known text (WKT).

In [45]:
import csv
from shapely.geometry import shape
from time import sleep

# Provide the location of the CSV file.
mbta_locs = 'camberville_t.csv'

# Open the file...
try:
    with open(mbta_locs, 'r') as infile:
        # ...and read its contents as a dictionary.
        reader = csv.DictReader(infile)
        # Iterate over the CSV rows...
        for line in reader:
            params = {'locations': [[line['X'], line['Y']]],
                          'profile': 'foot-walking',
                          'range_type': 'time',
                          'range': [600],
                          'smoothing': 10}
            try:
                request = ors.isochrones(**params)
                line['geom'] = shape(request['features'][0]['geometry']).wkt
                print(line['geom'])
                sleep(2)
            except Exception as err:
                pprint(err)
except FileNotFoundError:
    print(f'{mbta_locs} does not exist')

POLYGON ((-71.127904 42.374359, -71.127782 42.373757, -71.125147 42.368848, -71.124554 42.367938, -71.12429899999999 42.367684, -71.124137 42.367646, -71.123227 42.367732, -71.12224000000001 42.367897, -71.11667799999999 42.368036, -71.113326 42.368682, -71.111403 42.369224, -71.110163 42.370778, -71.110068 42.371126, -71.110195 42.375125, -71.11038600000001 42.377288, -71.110421 42.377425, -71.110427 42.377444, -71.110517 42.377679, -71.11171899999999 42.37827, -71.11555300000001 42.379995, -71.11627 42.380325, -71.119641 42.381553, -71.12 42.381587, -71.12463700000001 42.37988, -71.125452 42.379515, -71.127168 42.376799, -71.12788 42.374718, -71.127904 42.374359))
POLYGON ((-71.09629200000001 42.362991, -71.095298 42.360193, -71.095107 42.359828, -71.093642 42.358833, -71.09102799999999 42.357668, -71.0902 42.357616, -71.09001499999999 42.357632, -71.088888 42.358009, -71.08776 42.358386, -71.086803 42.358714, -71.08356499999999 42.359972, -71.08008 42.360486, -71.075638 42.361224, -

KeyboardInterrupt: 

However, we're still not writing the results out---below, we open an output file, create it with the same columns as the input with an addition `geom` column, write the headers to the output file, and write each row to the output. Kudos! You've built an isochrone tool that returns a CSV storing the isochrone geoemtries as Well-Known Text (WKT). QGIS can interpret this easily, as can PostGIS and other GISystems.

In [ ]:
import csv
from shapely.geometry import shape
from time import sleep

# Provide the location of the CSV file.
mbta_locs = 'camberville_t.csv'
out_file = 'camberville_t_iso.csv'

# Open the file...
try:
    with open(mbta_locs, 'r') as infile:
        with open(out_file, 'w') as outcsv:
            # ...and read its contents as a dictionary.
            reader = csv.DictReader(infile)
            field_names = reader.fieldnames + ['geom']
            writer = csv.DictWriter(outcsv, field_names)
            writer.writeheader()
            # Iterate over the CSV rows...
            for line in reader:
                params = {'locations': [[line['X'], line['Y']]],
                              'profile': 'foot-walking',
                              'range_type': 'time',
                              'range': [600],
                              'smoothing': 10}
                try:
                    request = ors.isochrones(**params)
                    line['geom'] = shape(request['features'][0]['geometry']).wkt
                    sleep(2)
                    writer.writerow(line)
                except Exception as err:
                    pprint(err)
except FileNotFoundError:
    print(f'{mbta_locs} does not exist')